# This notebook shows how to estimate a spot with a gaussian model.
* It is possible to fit a spot with one or several gaussians.
* Differents algorithmes are available.

In [1]:
%matplotlib notebook

import multiprocessing.pool
import pathlib

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc import Diagram
from laueimproc.io.download import get_samples  # gives access to the dataset

### Firstly, we have to create the diagrams from images
* To see all the ways to create a diagram, please have a look to the `init_diagram` notebook.
* Take care having enough RAM and swappiness to load all the images, otherwise it would bias the peak search time.

In [2]:
all_files = sorted(get_samples().iterdir())  # the list of all images path
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots()

## Fit by one gaussian

In [3]:
# very fast fit with "expectation maximization" algorithm
mean_em, cov_em, infodict_em = diagrams[0].fit_gaussian_em()  # like pca and barycenter
print("shape mean:", mean_em.shape)  # the position i, j of the center of the gaussian for each spot
print("shape cov:", cov_em.shape)  # the covariance matrix of the gaussian for each spot
assert infodict_em == {}  # empty because no parameters provided
*_, infodict_em = diagrams[0].fit_gaussian_em(aic=True, bic=True, eigtheta=True, log_likelihood=True)
print(sorted(infodict_em))  # more informations
*_, infodict_em = diagrams[0].fit_gaussian_em(photon_density=10900.0, tol=True)
print(sorted(infodict_em))  # mean standard deviation estimator

# fast fit with classical loss minimization
mean_mse, cov_mse, mag_mse, infodict_mse = diagrams[0].fit_gaussian(loss="mse", parallel=False)
print("shape mean:", mean_em.shape)  # the position i, j of the center of the gaussian for each spot
print("shape cov:", cov_em.shape)  # the covariance matrix of the gaussian for each spot

shape mean: torch.Size([135, 2])
shape cov: torch.Size([135, 2, 2])
['aic', 'bic', 'eigtheta', 'log_likelihood']
['tol']
rois shape torch.Size([135, 118, 84])
cost 1: 0.030207863077521324
cost 2: 0.030178802087903023
cost 3: 0.03014976717531681
cost n: 0.010735340416431427
cost n: 0.005694941151887178
cost n: 0.005610319785773754
cost n: 0.0049039521254599094
cost n: 0.004138077609241009
cost n: 0.003933510277420282
cost n: 0.003837284864857793
cost n: 0.0035880343057215214
cost n: 0.0033844925928860903
cost n: 0.0032344514038413763
cost n: 0.003084283322095871
cost n: 0.0029645187314599752
cost n: 0.0028938918840140104
cost n: 0.002822342561557889
cost n: 0.002759514143690467
cost n: 0.0026860488578677177
cost n: 0.0026253717951476574
cost n: 0.0025679883547127247
cost n: 0.0025183141697198153
cost n: 0.00246175448410213
cost n: 0.002416356699541211
cost n: 0.0023872186429798603
cost n: 0.0023646980989724398
cost n: 0.0023482604883611202
cost n: 0.0023211250081658363
cost n: 0.0023071

NotImplementedError: 